In [51]:
# public packages
import andes
import ams
import cvxpy as cp
import numpy as np
import gurobipy

# Load ams model

In [52]:
ssp = ams.load('ams_case123_REGCV1_v6.xlsx')

ssp.LDOPF.setup()

ssp.LDOPF.om.mdl.solve()

5.107460373258133

In [53]:
ssp.LDOPF.obj.v

5.107460373258133

In [54]:
ssp.LDOPF.om.vars

OrderedDict([('pg', Variable((11,))),
             ('pn', Variable((117,))),
             ('qg', Variable((11,))),
             ('qn', Variable((117,))),
             ('vsq', Variable((117,))),
             ('pl', Variable((116,))),
             ('ql', Variable((116,)))])

In [55]:
pg_ams = ssp.LDOPF.om.pg.value

pg_ams

array([0.05001063, 0.09001063, 0.0768937 , 0.02001063, 0.02401063,
       0.01401063, 0.01201063, 0.01801063, 0.01501063, 0.01601063,
       0.01701063])

In [56]:
ssp.LDOPF.pg.get_idx()

['PV_1',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9',
 'PV_10',
 'PV_11']

# Define new variables, constraints, and obj

In [57]:
constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]

In [58]:
constraints = constriants_ams

# Formulate new problem

pg = ['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

 SG: PV_1, PV_2, PV_3 (slack bus)

 IBR: PV_4, PV_5, ..., PV_11

In [59]:
# Cost data

c2 = np.array([0.01, 0.01, 0.02, 0, 0, 0, 0, 0, 0, 0, 0]) # gen quadratic cost
c1 = np.array([  20,   20,   30, 0, 0, 0, 0, 0, 0, 0, 0]) # gen linear cost

pg = ssp.LDOPF.om.pg

# pg_power = cp.power(pg, 2)
pg_power = pg**2

cost = cp.sum(c2 @ pg_power + c1 @ pg)

obj = cp.Minimize(cost)

In [60]:
ssp.LDOPF.c2.v

array([0.01, 0.01, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [61]:
# Create the optimization problem
prob = cp.Problem(obj, constraints)

In [62]:
# built-in solver: ECOS_BB
# prob.solve(solver=cp.ECOS_BB, verbose=True)
# prob.solve(solver=cp.ECOS_BB)
prob.solve(solver=cp.GUROBI)

5.110224580001787

In [63]:
ssp.LDOPF.c1

RParam: c1 <GCost>, v=[20. 20. 30.  0.  0.  0.  0.  0.  0.  0.  0.]

In [64]:
ssp.LDOPF.c2

RParam: c2 <GCost>, v=[0.01 0.01 0.02 0.   0.   0.   0.   0.   0.   0.   0.  ]

In [65]:
ssp.LDOPF.om.pg.value

[0.05, 0.09, 0.077, 0.02, 0.024, 0.014, 0.012, 0.018, 0.015, 0.016, 0.017]

In [66]:
sum(ssp.LDOPF.om.pg.value)

0.35300000000000015

In [67]:
# check lower limit of pg
ssp.LDOPF.pmin.v

array([0.025, 0.025, 0.025, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   ])

In [68]:
# check upper limit of pg
ssp.LDOPF.pmax.v

array([0.05 , 0.09 , 0.2  , 0.02 , 0.024, 0.014, 0.012, 0.018, 0.015,
       0.016, 0.017])

# 